# **Meerkat Call Classifier**

This code takes in examples of calls of different types, trains a conv net model to detect which type, then can run this on new data.

In [1]:
#from Jake

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import GlobalAveragePooling2D, Dense
import os
import numpy as np
import librosa
import scipy.signal as spy

#params
batch_size=10
clips_dir  = '/home/arianasp/meerkat_detector/clips'
call_types = ['cc','sn','ld','mov','agg','alarm','soc','hyb','unk','oth']
verbose = True
model_dir = '/home/arianasp/meerkat_detector/models'
model_name = 'test_classifier_diffsize.h5'
epochs = 10
steps_per_epoch = 1000
max_size = 512

#MAIN

inputs = Input(shape=(None, None,1))
x = inputs
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D()(x)

# this converts feature maps to 1D feature vectors by taking the channelwise average
# Also see GlobalMaxPooling2D and Flatten layers, which also transform to 1D vectors
# GlobalAveragePooling2D has some nice properties in terms of model regularization,
# which typically makes it the best choice for this transformation
x = GlobalAveragePooling2D()(x)  
x = Dense(64, activation='relu')(x)

N_CLASSES = 10
class_probabilites = Dense(N_CLASSES, activation='softmax')(x)

model = Model(inputs=inputs, outputs=class_probabilites)


# use categorical_crossentropy loss if your training data are one-hot encoded vectors
# and use sparse_categorical_crossentropy loss if your training data
# are integers indicating the class label (Keras one-hot encodes this internally for training)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy']) # same applies for sparse_categorical_accuracy

#Funcs to generate data
def generate_sample_call(files, clips_dir, max_size = 512, call_type = 'cc', call_types = ['cc','sn','ld','mov','agg','alarm','soc','hyb','unk','oth'], verbose=False):

    #hard coded parameters for now
    samprate=8000
    pad_len = 127

    #get indexes to files of type call_type (or sample from all calls if call_type is None)
    if(call_type is not None):
        idxs = np.where([((call_type in s) & ('_aud' in s)) for s in files])[0]
    else:
        idxs = np.where(['_aud' in s for s in files])[0]
        
    idx = np.random.choice(idxs)
    aud_file = files[idx]
    aud = np.load(clips_dir + '/' + aud_file)
    len_call = len(aud) - samprate*2
    if(len_call < 4096):
        offset = np.random.randint(samprate - 4096 + len_call, samprate)
    else:
        offset = samprate
    
    #len_call= 0
    #while(len_call < 240):
        #idx = np.random.choice(idxs)

        #get audio file
        #aud_file = files[idx]
    
        #read in audio file
        #aud = np.load(clips_dir + '/' + aud_file)
    
        #get call length
        #len_call = len(aud) - samprate*2
    
    #aud = aud[(samprate-pad_len):(samprate+pad_len + 4096)]
    
    #generate spectrogram
    #_,_,spec = spy.spectrogram(aud,fs=samprate,nperseg=255,noverlap=247,window='hanning')
    _,_,spec = spy.spectrogram(aud[(offset-pad_len):(offset+4096+pad_len)],fs=samprate,nperseg=255,noverlap=247,window='hanning')
    spec_norm = np.log(spec)
    
    #pad with 0's where no call is
    #spec_norm[:,int(len_call/8000.*1000):spec_norm.shape[1]] = 0

    #generate matrix for input to classifier
    X = np.transpose(spec_norm)
    
    #call type as integer (index in call_types vector)
    y = call_types.index(call_type)
    
    return X, y

def generate_batch(batch_size,clips_dir,call_types = ['cc','sn','ld','mov','agg','alarm','soc','hyb','unk','oth'],call_probs = None, verbose=False):
    
    #list files
    files = os.listdir(clips_dir)
    
    #create empty lists to hold spectrograms (input) and call labels (output)
    X_list = []
    y_list = []
    
    #probability of drawing each call type can be specified or if not specified is assumed to be uniform across all call types
    if(call_probs is not None):
        call_cumprobs = np.cumsum(call_probs)
    else:
        ncalls = len(call_types)
        call_cumprobs = np.arange(0,1,step=1./ncalls)+1./ncalls #equally spaced bins for drawing probabilities
        
    #generate samples
    for idx in range(batch_size):
        
        #select call type
        r = np.random.random(1)
        idx = np.where(call_cumprobs > r)[0][0]
        call_type = call_types[idx]
        X, y = generate_sample_call(files=files, clips_dir=clips_dir, call_type=call_type, call_types=call_types, verbose=verbose)
        
        #reshape for 2d conv
        X = X.reshape((X.shape[0],X.shape[1],1))
        #y = y.reshape((y.shape[0],y.shape[1],1))
        
        #append to list
        X_list.append(X)
        y_list.append(y)
    
    #convert to stack
    X = np.stack(X_list)
    y = np.stack(y_list)
    return (X, y)

#Data generators 
def data_generator(clips_dir,batch_size=10,call_types = ['cc','sn','ld','mov','agg','alarm','soc','hyb','unk','oth'],call_probs = [1./7,1./7,1./7,1./7,1./7,1./7,1./7,0,0,0],verbose=False):
    while True:
        yield generate_batch(batch_size,clips_dir,call_types,call_probs,verbose=verbose)

        
#fit model
model.fit_generator(data_generator(clips_dir = clips_dir,batch_size = 4), epochs=epochs, use_multiprocessing=True, workers=16, steps_per_epoch=steps_per_epoch)

#Save fitted model
print('Saving model as: ' + model_name)
model.save(filepath=model_dir + '/' + model_name) 

Using TensorFlow backend.


Epoch 1/10


/home/arianasp/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


  81/1000 [=>............................] - ETA: 18s - loss: 2.1308 - sparse_categorical_accuracy: 0.2284

ResourceExhaustedError: OOM when allocating tensor with shape[4,32,510,126] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, data_format="NCHW", ksize=[1, 1, 2, 2], padding="VALID", strides=[1, 1, 2, 2], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/Relu, max_pooling2d_1/MaxPool, training/Adam/gradients/conv2d_2/convolution_grad/Conv2DBackpropInput)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
import os
import numpy as np
import scipy.signal as spy
import matplotlib.pyplot as plt
files = os.listdir(clips_dir)
for i in range(20):
    X ,y = generate_sample_call(files,clips_dir,call_type='mov')
    print(X.shape)
#plt.imshow(X,aspect='auto')
X, y = generate_batch(10,clips_dir)
plt.imshow(X[0,:,:,0].T)
print(y)
print(call_types[y[0]])

In [ ]:
X,y = generate_batch(clips_dir=clips_dir,batch_size=1,call_probs = [1./7,1./7,1./7,1./7,1./7,1./7,1./7,0,0,0])
plt.imshow(X[0,:,:,0].T)
print(call_types[y[0]])

out= model.predict(X)
print(call_types[np.argmax(out[0])])

In [ ]:
print(range(ncalls))

In [ ]:
aud=np.load('/home/arianasp/meerkat_detector/clips/agg1532_aud.npy')
print(len(aud))